In [1]:
# Manipular os arquivos de vídeo 
import cv2
import os

# Abrir arquivos com o explorador
import tkinter as tk
from tkinter import filedialog

In [2]:
# O tkinter é utilizado para a criação de interfaces gráficas
# como janelas, botões, caixas de entrada, etc.
# Neste caso, ele é utilizado para criar a janela que permite
# a escolha do arquivo de vídeo.

root = tk.Tk() # Cria a instância do tkinter
root.withdraw()  # Oculta a janela principal da interface, mantendo apenas o necessário
root.attributes('-topmost', True)  # Força a janela a ficar no topo
root.update()  # Atualiza a janela para aplicar as mudanças

# Trecho responsável por abrir uma janela para a escolha de um arquivo (seu path)

video_path = filedialog.askopenfilename( # A função do tkinter abre o explorador de arquivos
    title="Selecione um vídeo", # Nome da janela a ser aberta
    filetypes=[("Arquivos de vídeo", "*.mp4;*.avi;*.mov;*.mkv")], # Define quais arquivos podem ser escolhidos
    parent=root # Define a janela oculta do tkinter (root) como a janela pai do explorador, com isso
                # as alterações em root afetam a janela do explorador (como manter ela em primeiro plano)
    )

root.destroy() # Elimina a janela após a escolha do arquivo

# Exibe ao usuário do programa se algum arquivo foi ou não escolhido
# de acordo com a presença de um caminho na variável video_path

if not video_path:
    print("Nenhum arquivo selecionado.")
else:
    print(f"Arquivo selecionado: {video_path}")

# Cria uma pasta (caso não exista) para alocar os frames salvos

output_folder = "frames"
os.makedirs(output_folder, exist_ok=True)



Arquivo selecionado: C:/Users/marci_plgx30x/Desktop/Arquivos/Mestrado 3101/Videos/WhatsApp Video 2025-01-28 at 13.50.50.mp4


In [20]:
# Abrindo o vídeo
cap = cv2.VideoCapture(video_path)

# Encerra a aplicação caso não seja possível executar o vídeo

if not cap.isOpened():
    print("Erro ao abrir o vídeo.")
    exit()

frame_index = 0 # Inicia a ordenação dos frames sendo o primeiro frame = 0
selected_frames = [] # Lista que descreve a posição dos frames que devem ser salvos
save_msg = ""  # Mensagem a ser exibida na imagem, quando ela for salva
counter_msg = 0 # Inicializa o contador para a mensagem de save

# Executa em loop a exibição dos frames do vídeo

while True:

    # O trecho pula a visualização para o frame definido por 'frame index', sendo:

    # cap -> é um objeto do tipo cv2, que representa o vídeo carregado.
    # CAP_PROP_POS_FRAMES -> define a posição do próximo frame a ser lido
    # frame_index -> número do frame que desejamos acessar

    # OBS: o comando cv2.CAP_PROP_POS_FRAMES é uma constante do openCV que permite
    # posicionar o ponteiro de leitura de vídeo em um frame específico, antes de 
    # realizar a leitura do mesmo.

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

    # O trecho realiza a leitura do próximo frame disponível do vídeo
    # serão retornadas duas variáveis: ret -> indica se a leitura foi bem sucedida (bool)
    # e frame retorna a imagem do frame como matriz numpy

    # cap.read() faz a leitura do frame definido em cap.set()

    ret, frame = cap.read()

    # Se a leitura não for bem sucedida, ou seja, se ret == False
    # o programa é encerrado

    if not ret:
        print("Fim do vídeo ou erro na leitura.")
        break

    # Mostra o frame atual

    display_frame = frame.copy() # É realizado uma cópia do frame a ser exibido (a fim de não afetar o original)

    # O seguinte comando tem a função de inserir um texto na imagem a ser exibida
    # neste caso, o texto exibe qual o frame que está sendo analisado. O comando
    # tem como seguintes parâmetros:

    # 1 - display_frame: A imagem onde o texto será desenhado
    # 2 - f"Frame: {frame_index}": O texto que será desenhado
    # 3 - (10, 30): Coordenadas do ponto de início do texto na imagem
    # 4 - cv2.FONT_HERSHEY_SIMPLEX: Tipo de fonte do texto
    # 5 - 1: Define o tamanho do texto, sendo 1 o tamanho padrão
    # 6 - (0, 255, 0): Cor do texto, em formato RGB (R,G,B)
    # 7 - 2: Define a espessura do texto a ser escrito

    cv2.putText(display_frame, 
                f"Frame: {frame_index}", 
                (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0, 255, 0), 
                2)
    

    # Exibe o texto de sucesso do salvamento por um tempo pré-determinado
    if counter_msg > 0:

            cv2.putText(display_frame, 
                save_msg, 
                (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0, 255, 0), 
                2)
            
            counter_msg -= 1  # Reduzir o contador para remover a mensagem depois de alguns frames


    # Exibe o frame, já com o texto escrito
    
    cv2.imshow("Frame Selector", display_frame)

    # O seguinte trecho de código permite ao usuário ter controle
    # sobre o programa, sendo possível escolher e salvar frames

    # o comando cv2.waitKey(0) & 0xFF é utilizado para capturar uma tecla
    # pressionada pelo usuário enquanto uma janela de exibição cv2 está aberta,
    # além disso, vai definir o tempo de cada iteração do loop,nete caso
    # o argumento 30 passado para a função waitKey() indica que o programa 
    # deve aguardar 30ms até que uma tecla seja pressionada, com isso:

    # 1 iteração leva 30ms, 30 iterações levam 900ms, aproximadamente 1 segundo. 
    # consequentemente a execução opera a 30 fps

    # obs: o retorno da função cv2.waitKey() pode conter 32 bits, por mais que 
    # apenas os 8 bits finais sejam necessários para representar a tecla pressionada
    # com isso, utilizamos a operação & 0xFF para garantir que apenas os 8 bits menos
    # significativos sejam mantidos. 

    key = cv2.waitKey(20) & 0xFF

    # O seguinte trecho de código busca executar uma determinada ação, de acordo com a tecla
    # pressionada. vale ressaltar que a função ord() converte o caractere em uma valor
    # correspondente da tabela ASCII

    if key == ord('d'):  
        frame_index += 1 # Avançar um frame

    elif key == ord('a'): 
        frame_index = max(0, frame_index - 1) # Retroceder um frame, retrocedendo no máximo até o frame inicial = 0

    elif key == ord('s'):  

        # O trecho a seguir define o nome do frame a ser salvo, juntamente com o nome da pasta onde o frame será salvo, caso
        # contrário o frame seria salvo no mesmo diretório onde o arquivo.py do programa se encontra

        filename = os.path.join(output_folder, f"frame_{frame_index:04d}.png")
    
        cv2.imwrite(filename, frame) # Salva o frame selecionado com o nome e o caminho definido em "filename"

        selected_frames.append(frame_index) # Guarda uma lista com os frames salvos, atualizando o frame salvo
       
        save_msg = f"Frame {frame_index} salvo em com sucesso"  # Atualizar a mensagem

        counter_msg = 20 # bandeira que permite exibir a mensagem de sucesso ao salvar o frame
                         # este contador controla por quantos frames a mensagem será visível

    elif key == ord('q'): 
        break # Encerra a execução do programa

# Ao fim da execução o arquivo de vídeo é liberado da memória e todas
# as janelas abertas são encerradas
cap.release()
cv2.destroyAllWindows()